In [1]:
import pickle
import dynet
import os
import numpy as np
%run utils.ipynb

In [2]:

if __name__ == '__main__':
    (vocab, net_properties) = pickle.load(open('../data/vocab_net.data', 'rb'))
    network = Network(vocab, net_properties)
    network.load('../data/net.model')

    writer = open('../data/predictions.data', 'w')
    feature_list = pickle.load(open( "../data/test.data", "rb" ))

    correct = 0
    shiftstars, tps, fps, fns, shifts, tpss, fpss, fnss = 0, 0, 0, 0, 0, 0, 0, 0
    for feature_set in feature_list:
        pred = network.decode(feature_set[:-1])
        writer.write("ground truth: " + feature_set[-1] + ", prediction: " + pred + "\n\n")

        # count shift stars
        if pred == "shift star":
            if feature_set[-1] == "shift star":
                shiftstars += 1
                tpss += 1
            else:
                fpss += 1
        if feature_set[-1] == "shift star" and pred != "shift star":
            fnss += 1
            shiftstars += 1

        # count shifts
        if pred == "shift":
            if feature_set[-1] == "shift":
                shifts += 1
                tps += 1
            else:
                fps += 1
        if feature_set[-1] == "shift" and pred != "shift":
            fns += 1
            shifts += 1

        # total accuracy
        if pred == feature_set[-1]:
            correct += 1


    print("accuracy: " + str(float(correct)/len(feature_list)))

    print("----------------------------------------")

    precision_ss = tpss / (tpss + fpss)
    recall_ss = tpss / (tpss + fnss)
    print("star precision: " + str(precision_ss))
    print("star recall: " + str(recall_ss))
    print("total stars: " + str(shiftstars))
    print("star F1: " + str( 2*(precision_ss * recall_ss) / (precision_ss + recall_ss) ))

    print("----------------------------------------")

    precision_s = tps / (tps + fps)
    recall_s = tps / (tps + fns)
    print("shift precision: " + str(precision_s))
    print("shift recall: " + str(recall_s))
    print("total shifts: " + str(shifts))
    print("shift F1: " + str( 2*(precision_s * recall_s) / (precision_s + recall_s) ))

    writer.close()

accuracy: 0.836
----------------------------------------
star precision: 0.8115942028985508
star recall: 0.691358024691358
total stars: 162
star F1: 0.7466666666666666
----------------------------------------
shift precision: 0.8550681683461766
shift recall: 0.941579634464752
total shifts: 3064
shift F1: 0.8962410686548617


In [7]:
def remove_star(s):
    s = s.split()
    s = list(filter(lambda x: '*' not in x, s))
    return ' '.join(s)

def action(b, s, p):
    error = None

    if p.split()[0] == 'shift':
        if len(p.split()) > 1 and p.split()[1] == 'star':
            s.append(Node('*'))
            
        # normal shift
        try: s.append(b.pop(0))
        except: error = 'pop on empty buffer'

    elif p.split()[0] == 'unary':
        n = Node(clean(p.split()[1]))
        try:
            n.l = s.pop()
            s.append(n)
        except: error = 'unary on empty stack'

    else: # p.split()[0] == 'binary':
        n = Node(clean(p.split()[1]))
        try:
            n.r, n.l = s.pop(), s.pop()
            s.append(n)
        except: error = 'binary on insufficient stack'

    return b, s, error


if __name__ == '__main__':
    # load the network
    (vocab, net_properties) = pickle.load(open('../data/vocab_net.data', 'rb'))
    network = Network(vocab, net_properties)
    network.load('../data/net.model')

    # open treebank for testing
    treepath = "../treebank/treebank_3/parsed/mrg/wsj/24"

    # open file and save as one large string
    text = ""
    for filename in os.listdir(treepath):
        if filename.startswith('.'):
            continue
        with open(treepath + "/" + filename, 'r') as f:
            text += f.read().replace('\n', '')

    tree_string_list = []
    s = []
    start = 0
    for i in range(len(text)):
        if text[i] == "(":
            s.append("(")
        elif text[i] == ")":
            s.pop()
            if not s:
                tree_string_list.append(text[start : i + 1])
                start = i + 1

    # turn tree strings into tree_list
    tree_list = []
    for t in tree_string_list:
        tree_list.append((parse_tree(t[1:-1])))

    # use inorder traveral to generate sentences from trees
    sentences = []
    for t in tree_list:
        sentences.append(remove_star(inorder_sentence(t).lstrip()))

    # testing
    with open('../data/tree_pred.txt', 'w') as outfile:
        for s, t in zip(sentences, tree_list):
            s = [clean(x) for x in s.split()]
            
            #debug
            print(' '.join(s) + '\n') # print sentence
            outfile.write(' '.join(s) + '\n\n')

            # construct tree
            buff = list(map(Node, s))
            stack = []
            infinite_loop_count = 0 # terminate after 100 moves
            while buff or len(stack) > 1: # end when buff consumed & stack has tree

                # cast to string and predict
                stack, buff = list(map(tree_to_str, stack)), list(map(tree_to_str, buff))
                try: f = extract_features(datum(stack, buff, None))
                except: 
                    print('feature extraction error')
                    break
                    

                pred = network.decode(rearrange([0] + f)[:-1])
                # outfile.write(str(f) + ' ' +  pred + '\n')

                # cast back to Node and complete action
                stack, buff = list(map(Node, stack)), list(map(Node, buff))
                buff, stack, error = action(buff, stack, pred)
                if error:
                    # outfile.write(error + '\n')
                    print('Error: ' + error)
                    outfile.write('Error: '+ error + '\n')
                    break
                print(pred + '\n' + stack_to_str(stack) + '\n')
                outfile.write(pred + '\n' + stack_to_str(stack) + '\n\n')
                
                infinite_loop_count += 1
                if infinite_loop_count >= 150: 
                    print('infinite loop error')
                    break
                
            print('GROUND TRUTH:\n' + tree_to_str(t) + '\n')
            print('-------------------end of sentence-----------------\n')
            outfile.write('GROUND TRUTH:\n' + tree_to_str(t) + '\n\n')
            outfile.write('-------------------end of sentence-----------------\n\n')



The economy 's temperature will be taken from several vantage points this week , with readings on trade , output , housing and inflation .

shift
[ The]

unary DT
[ (DT The)]

shift
[ (DT The),  economy]

unary NN
[ (DT The),  (NN economy)]

shift
[ (DT The),  (NN economy),  's]

unary POS
[ (DT The),  (NN economy),  (POS 's)]

binary NP
[ (DT The),  (NP (NN economy) (POS 's))]

binary NP
[ (NP (DT The) (NP (NN economy) (POS 's)))]

shift
[ (NP (DT The) (NP (NN economy) (POS 's))),  temperature]

unary VBD
[ (NP (DT The) (NP (NN economy) (POS 's))),  (VBD temperature)]

shift
[ (NP (DT The) (NP (NN economy) (POS 's))),  (VBD temperature),  will]

unary MD
[ (NP (DT The) (NP (NN economy) (POS 's))),  (VBD temperature),  (MD will)]

shift
[ (NP (DT The) (NP (NN economy) (POS 's))),  (VBD temperature),  (MD will),  be]

unary VB
[ (NP (DT The) (NP (NN economy) (POS 's))),  (VBD temperature),  (MD will),  (VB be)]

shift
[ (NP (DT The) (NP (NN economy) (POS 's))),  (VBD temperature),  (MD 


unary CD
[ (NP (DT The) (NP (NN trade) (NN gap))),  (VBZ is),  (VBN expected),  (TO to),  (VB widen),  (NP (PP (TO to) (NP (RB about) (NP ($ $) (NP (CD 9) (NP (NONE billion))))))),  (IN from),  (NP (NNP July) (POS 's)),  ($ $),  (CD 7.6),  (CD billion)]

binary NP
[ (NP (DT The) (NP (NN trade) (NN gap))),  (VBZ is),  (VBN expected),  (TO to),  (VB widen),  (NP (PP (TO to) (NP (RB about) (NP ($ $) (NP (CD 9) (NP (NONE billion))))))),  (IN from),  (NP (NNP July) (POS 's)),  ($ $),  (NP (CD 7.6) (CD billion))]

binary NP
[ (NP (DT The) (NP (NN trade) (NN gap))),  (VBZ is),  (VBN expected),  (TO to),  (VB widen),  (NP (PP (TO to) (NP (RB about) (NP ($ $) (NP (CD 9) (NP (NONE billion))))))),  (IN from),  (NP (NNP July) (POS 's)),  (NP ($ $) (NP (CD 7.6) (CD billion)))]

binary NP
[ (NP (DT The) (NP (NN trade) (NN gap))),  (VBZ is),  (VBN expected),  (TO to),  (VB widen),  (NP (PP (TO to) (NP (RB about) (NP ($ $) (NP (CD 9) (NP (NONE billion))))))),  (IN from),  (NP (NP (NNP July) (POS 's))


shift
[ (NP (DT That) (NN gain)),  (VBD was),  (VBG being),  (NP (NONE cited)),  (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))),  (NP (DT the) (NN stock)),  market]

unary NN
[ (NP (DT That) (NN gain)),  (VBD was),  (VBG being),  (NP (NONE cited)),  (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))),  (NP (DT the) (NN stock)),  (NN market)]

binary NP
[ (NP (DT That) (NN gain)),  (VBD was),  (VBG being),  (NP (NONE cited)),  (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))),  (NP (NP (DT the) (NN stock)) (NN market))]

binary PP
[ (NP (DT That) (NN gain)),  (VBD was),  (VBG being),  (NP (NONE cited)),  (PP (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))) (NP (NP (DT the) (NN stock)) (NN market)))]

binary VP
[ (NP (DT That) (NN gain)),  (VBD was),  (VBG being),  (VP (NP (NONE cited)) (PP (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))) (NP (NP (DT the) (NN stock)) (NN market))))]

binary VP
[ (NP (DT That) (NN gain)),  (VBD was),  (VP (VBG being) (VP (NP (NONE cited)) (PP (PP

unary ,
[ (NP (DT That) (NN gain)),  (VP (VBD was) (VP (VBG being) (VP (NP (NONE cited)) (PP (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))) (NP (NP (DT the) (NN stock)) (NN market)))))),  (VBD was),  (ADJP (RB down) (RB early)),  (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (PP (IN in) (NP (NP (NNP Friday) (POS 's)) (NNP session))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))]

unary ,
[ (NP (DT That) (NN gain)),  (VP (VBD was) (VP (VBG being) (VP (NP (NONE cited)) (PP (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))) (NP (NP (DT the) (NN stock)) (NN market)))))),  (VBD was),  (ADJP (RB down) (RB early)),  (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (PP (IN in) (NP (NP (NNP Friday) (POS 's)

unary ,
[ (NP (DT That) (NN gain)),  (VP (VBD was) (VP (VBG being) (VP (NP (NONE cited)) (PP (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))) (NP (NP (DT the) (NN stock)) (NN market)))))),  (VBD was),  (ADJP (RB down) (RB early)),  (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (PP (IN in) (NP (NP (NNP Friday) (POS 's)) (NNP session)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))]

unary ,
[ (NP (DT That) (NN gain)),  (VP (VBD was) (VP (VBG being) (VP (NP (NONE cited)) (PP (PP (IN as) (NP (NP (DT a) (NN reason)) (NONE 0))) (NP (NP (DT the) (NN stock)) (NN market)))))),  (VBD was),  (ADJP (RB down) (RB early)),  (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (, (,

...


unary NN
[ (NP (DT The) (NN pressure)),  (S (VP (VBZ is) (NP (RB too) (NP (RB much) (NP (NNP for) (NP (NNP Zsa) (NNP Zsa)))))) (S (, ,) (WP who))),  (VBP slaps),  (DT a),  security,  (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN guard)))))))))))))))))))))))))))))))))))))))))))))))))))))))]

unary NN
[ (NP (DT The) (NN pressure)),  (S (VP (VBZ is) (NP (RB too) (NP (RB much) (NP (NNP for) (NP (NNP Zsa) (NNP Zsa)))))) (S (, ,) (WP who))),  (VBP slaps),  (DT a),  security,  (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN guard))))))))))))))))))))))))))))))))))))))))))))))))))))))))]

unary NN
[ (NP (DT The) (NN pressure)),  (S (VP (VBZ is) (NP (

unary NN
[ (NP (DT The) (NN pressure)),  (S (VP (VBZ is) (NP (RB too) (NP (RB much) (NP (NNP for) (NP (NNP Zsa) (NNP Zsa)))))) (S (, ,) (WP who))),  (VBP slaps),  (DT a),  security,  (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN guard)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))]

unary NN
[ (NP (DT The) (NN pressure)),  (S (VP (VBZ is) (NP (RB too) (NP (RB much) (NP (NNP for) (NP (NNP Zsa) (NNP Zsa)))))) (S (, ,) (WP who))),  (VBP slaps),  (DT a),  security,  (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN (NN

binary NP
[ (NP (NP (PRP He)) (NP (NNP backflips) (NP (IN into) (NP (NP (DT a) (NP (NN desktop) (NP (NN computer) (NN terminal)))) (NP (, ,) (NP (WDT which) (NP (NN explodes) (NP (, ,) (NP (IN covering) (NP (NP (NP (NN Huntz) (NP (NNP Hall) (POS 's))) (NN face)) (PP (IN with) (ADVP (RB microchips)))))))))))))]

shift
[ (NP (NP (PRP He)) (NP (NNP backflips) (NP (IN into) (NP (NP (DT a) (NP (NN desktop) (NP (NN computer) (NN terminal)))) (NP (, ,) (NP (WDT which) (NP (NN explodes) (NP (, ,) (NP (IN covering) (NP (NP (NP (NN Huntz) (NP (NNP Hall) (POS 's))) (NN face)) (PP (IN with) (ADVP (RB microchips))))))))))))),  .]

unary VBD
[ (NP (NP (PRP He)) (NP (NNP backflips) (NP (IN into) (NP (NP (DT a) (NP (NN desktop) (NP (NN computer) (NN terminal)))) (NP (, ,) (NP (WDT which) (NP (NN explodes) (NP (, ,) (NP (IN covering) (NP (NP (NP (NN Huntz) (NP (NNP Hall) (POS 's))) (NN face)) (PP (IN with) (ADVP (RB microchips))))))))))))),  (VBD .)]

Error: pop on empty buffer
GROUND TRUTH:
 (S (NP-SB

unary NNP
[ (IN In),  (NP (DT an) (NN attempt)),  (TO to),  (VB appease),  (NP (NNP U.S.) (NP (NNS regulators) (NP (NN ') (NN concern)))),  (IN over),  (NP (DT a) (NP (NN Goldsmith) (NN acquisition))),  (IN of),  (NP (NNP Farmers)),  (, ,),  (NP (NNP Sir) (NNP James)),  (IN in),  (NNP August)]

shift
[ (IN In),  (NP (DT an) (NN attempt)),  (TO to),  (VB appease),  (NP (NNP U.S.) (NP (NNS regulators) (NP (NN ') (NN concern)))),  (IN over),  (NP (DT a) (NP (NN Goldsmith) (NN acquisition))),  (IN of),  (NP (NNP Farmers)),  (, ,),  (NP (NNP Sir) (NNP James)),  (IN in),  (NNP August),  agreed]

unary NNS
[ (IN In),  (NP (DT an) (NN attempt)),  (TO to),  (VB appease),  (NP (NNP U.S.) (NP (NNS regulators) (NP (NN ') (NN concern)))),  (IN over),  (NP (DT a) (NP (NN Goldsmith) (NN acquisition))),  (IN of),  (NP (NNP Farmers)),  (, ,),  (NP (NNP Sir) (NNP James)),  (IN in),  (NNP August),  (NNS agreed)]

unary NP
[ (IN In),  (NP (DT an) (NN attempt)),  (TO to),  (VB appease),  (NP (NNP U.S.) (NP

KeyboardInterrupt: 